In [50]:
# Import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback
import random

# Create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Load the intents JSON file
nltk.download('punkt')
nltk.download('wordnet')

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("intents.json").read()
intents = json.loads(data_file)

# Preprocess the data
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)  # Tokenize each word
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize, lowercase, and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Create training data
training = []
output_empty = [0] * len(classes)

# Create bag of words for each pattern
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

# Split data into training and validation sets
train_x, val_x, train_y, val_y = train_test_split(list(training[:, 0]), list(training[:, 1]), test_size=0.2, random_state=42)

# Save words and classes for later use
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

# Build the model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
from tensorflow.keras.optimizers import Adam
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Define F1 Score callback
class F1ScoreCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(np.array(val_x))
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = np.argmax(np.array(val_y), axis=1)

        # Calculate F1 Score
        f1 = f1_score(y_true, y_pred_classes, average='weighted')
        print(f"\nF1 Score for epoch {epoch + 1}: {f1:.4f}")

# Train the model       
hist = model.fit(
    np.array(train_x), np.array(train_y),
    validation_data=(np.array(val_x), np.array(val_y)),
    epochs=200, batch_size=32, verbose=1,
    callbacks=[F1ScoreCallback()]
)

# Save the model
model.save('model.h5', hist)
print("\nModel Created Successfully!")


Epoch 1/200


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\samue\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/stepp - accuracy: 0.0312 - loss: 3.882

F1 Score for epoch 1: 0.0093
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.0230 - loss: 3.9037 - val_accuracy: 0.0233 - val_loss: 3.8360
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step  - accuracy: 0.0625 - loss: 3.866

F1 Score for epoch 2: 0.0116
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0342 - loss: 3.8781 - val_accuracy: 0.0233 - val_loss: 3.8327
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0938 - loss: 3.798

F1 Score for epoch 3: 0.0116
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.0472 - loss: 3.8474 - val_accuracy: 0.0233 - val_loss: 3.8284
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0625 - loss: 3.818

F1 Score for epoch 4: 0.0155
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0511 - loss: 3.8230 - val_accuracy: 0.0233 - val_loss: 3.8246
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0938 - loss: 3.755

F1 Score for epoch 


Model Created Successfully!
